# Récupération d'une base de parties via l'API

*Remarque : Des fichiers csv vont être créés au fur et mesure en utilisant nos programmes et ils seront nécessaires pour que les programmes suivants puissent fonctionner*

### Étape 1 : Obtention du summonerId des joueurs classés diamant dans le mode compétitif de League of Legends

Il est nécessaire d'avoir un ***api_key*** pour envoyer utiliser l'API de Riot, cependant cet ***api_key expire après 24h***, ainsi pour faire tourner nos programmes, il est necessaire de l'actualiser, auquel cas l'erreur *key_error* sera renvoyée. Pour cela, il faut se rendre sur le site https://developer.riotgames.com/. Cependant pour pouvoir l'utiliser il faut s'identifier sur la plateforme, suite à quoi on pourra récupérer notre *api_key*.

In [1]:
#Entrez votre api_key actualisé, ou envoyez nous un mail si vous voulez que l'on vous en envoie un opérationnel:
api_key = "RGAPI-cdee5bdb-9d85-46ae-a324-4ebaca02da63"

Aussi, on ne peut uniquement réaliser que 20 requêtes par seconde et 100 requêtes toutes les 2 minutes.

L'API de LoL nous permet de récupérer le ***summonerId*** des joueurs d'un certain rang (ici on s'intéresse aux joueurs de rang diamant) : https://developer.riotgames.com/apis#league-exp-v4/GET_getLeagueEntries.

*Le programme suivant met une quinzaine de secondes à s'executer*

In [2]:
from summIdExtractor import summ_ID_puller
summ_ID_puller(api_key)

In [28]:
pd.read_csv('summID.csv', index_col = 0).head(3)

,Summoner ID
0.0,zcIDc81ls-pmJFCUO6acjPLtzO3G0mHBDb-FQMO-HZ1lM8Ra
1.0,nRHMygxa16fFUqgsgyvW6vmBSyaXtEis0avFIrMFFPt0lF0W
2.0,HHKp0lulBgCTuX_Yj5wR4LoFq97CUuiB0ss45oz0J9vHWO-v


Avec la boucle ci-dessus, nous avons stocké sous forme de fichier csv plus de 15000 identifiants de joueurs.

Cependant, comme dit plus tôt, on ne peut réaliser que 20 requêtes toutes par seconde, on applique donc un temps de pause entre chaque ittération de notre boucle.

Nous avons maintenant une liste des summonerId de nombreux joueurs, on va maintenant chercher à obtenir leur historique pour avoir une base de données de parties jouées, base sur laquelle on va travailler au cours de ce projet. Cepdandant, pour accéder à l'historique des parties, l'API de Riot requière un autre identifiant unique aux joueurs, le PUUID, ce que nous allons à présent récupérer.

### Étape 2 : Convertir les summonerId en PUUID

*Essayez d'attendre 2 minutes après l'utilisation du programme précedent car les 100 requêtes disponibles on déjà été utilisées*

Avec ce programme, on ne récupère que les PUUID des 100 premières personnes de ma liste précedente. En effet, lorsque nous allons chercher à récupérer l'historique de nos joueurs plus tard, nous allons voir que le nombre de requêtes vers l'API va rapidement augmenter et les temps d'executions atteindront plusieurs heures.

*Le programme suivant met une quinzaine de secondes à s'executer*

In [ ]:
from puuidConvertor import acct_ID_puller
accountID_list = acct_ID_puller(api_key)

In [ ]:
df = pd.DataFrame(accountID_list, columns=["AccountPUUID"])
df.to_csv('accountPUUID.csv', mode='x')

In [24]:
pd.read_csv('accountPUUID.csv',index_col = 0).head(3)

,AccountPUUID
0,XFhANjotWKg0W6cbFgevEEfMffiNggo8Ibef4ics8gUzMk...
1,EQNlOi64ynjI6NzBs6rHPKl95ssmPaE_MsiS_-aQkJPeiA...
2,AOxaU0kXvA45iYOMk2iibcqYlzMQk1KNOUpmr119HDJEYw...


### Étape 3 : Récupération de l'ID de matchs"

A présent nous allons commencer à nous créer une base de données de parties. Dans un premier temps, pour chaque joueur de notre base, nous allons récupérer son historique de partie sous la forme d'une liste d'ID avec laquelle on pourra dans un second temps obtenir les données de parties.

In [ ]:
from gameIdExtractor import match_ID_puller
df_gameId = match_ID_puller(api_key)

In [ ]:
df_gameId.to_csv('MatchId.csv', mode='x')

In [21]:
pd.read_csv('MatchId.csv',index_col = 0).head(3)

,Janvier,Février,Mars,Avril,Mai,Juin,Juillet,Aout,Septembre,Octobre,Novembre
0,EUW1_5697047365,EUW1_5747768505,EUW1_5797885608,EUW1_5832114947,EUW1_5893371604,EUW1_5946522890,EUW1_5954168607,EUW1_6032966424,EUW1_6084406916,EUW1_6112441706,EUW1_6137136125
1,EUW1_5691273837,EUW1_5724403495,EUW1_5797790240,EUW1_5832005330,EUW1_5893285573,EUW1_5946440329,EUW1_5954221246,EUW1_6032881787,EUW1_6084349783,EUW1_6122839121,EUW1_6137090066
2,EUW1_5691127938,EUW1_5710233832,EUW1_5797830988,EUW1_5831830238,EUW1_5893210267,EUW1_5945206205,EUW1_5954127808,EUW1_6031429345,EUW1_6083012143,EUW1_6122825786,EUW1_6137045074


### Étape 4 : Mise en place de la base de données

Enfin, on commence à mettre en place la base de données qui sera ensuite complétée avec du scraping.

Dans un premier temps, on retire les informations suivantes : les *pseudos* des joueurs dans chaque partie, le *champion* qu'ils jouent, et l'équipe qui remporte la partie.

In [4]:
from gameData import *

**La cellule suivants requièrent un temps d'executions de 5h à 9h**

In [ ]:
nb_parties = pd.read_csv("MatchId.csv").shape[0]
df_oct = match_data_oct(nb_parties)
df_nov = match_data_nov(nb_parties)

On obtient alors des tableaux dans le format suivant

In [22]:
df_oct.head(3)

,Blue Top,Blue Jungle,Blue Mid,Blue Carry,Blue Support,Red Top,Red Jungle,Red Mid,Red Carry,Red Support,Outcome
0,"[TripleBen, Nasus]","[Dark Memer, Zac]","[CroKette, Lux]","[ŃIRVANA, Kaisa]","[Bioniks1, Soraka]","[monkaS, Gangplank]","[N0EQUAL, Rengar]","[CG Kinneke, Yasuo]","[Bendåx, Ashe]","[Jonperi, Maokai]",Victoire des Rouges
1,"[but ok, Viego]","[Kanuera, Khazix]","[Meep Enjooyer, Viktor]","[tjfgx, Twitch]","[Vaati73, Yuumi]","[Lewite, Vayne]","[raymmondd, Nidalee]","[tuctuc1, Akali]","[qfxnhfzkl, Ezreal]","[Bingu, Thresh]",Victoire des Bleus
2,"[Lewite, Vayne]","[Lundqvistt, Volibear]","[Chieurdepatate, Swain]","[qfxnhfzkl, Twitch]","[spray em 50, Amumu]","[Sk8 The Infinity, Sejuani]","[ChemtrailsOverUs, Singed]","[Sekoiya, Yasuo]","[juicy ladyboy, Jinx]","[lοve too hard, Lulu]",Victoire des Bleus


In [6]:
df_final_oct.to_csv('MatchsOctobre.csv', mode='x')
df_final_nov.to_csv('MatchsNovembre.csv', mode='x')

#### Petite apparté remise en page

Sinon, avec les fichiers *MatchsOctobre.csv* et *MatchsOctobre.csv*, on peut exécuter les programmes suivant qui vont refaire la mise en page de nos DataFrame et les rendres plus lisibles

In [11]:
df_oct = pd.read_csv("MatchsOctobre.csv",index_col = 0)
for col in df_oct.columns[:-1]:
    df_oct[col] = df_oct[col].apply(lambda x: eval(x))
df_nov = pd.read_csv("MatchsNovembre.csv",index_col = 0)
for col in df_nov.columns[:-1]:
    df_nov[col] = df_nov[col].apply(lambda x: eval(x))

In [12]:
df_lecture_oct = convert_df(df_oct)
df_lecture_nov = convert_df(df_nov)

Je vous laisse juger de la pertinence de cette remise en page, mais je concède que ce refaire la mise en page du DataFrame sachant que l'on va travailler sur la première version a très peu d'utilité.

In [23]:
df_lecture_oct.head(4)

Blue Top Blue Jungle       Blue Mid Blue Carry  \
Partie   Infos                                                       
Partie 1 Pseudo    TripleBen  Dark Memer       CroKette    ŃIRVANA   
         Champion      Nasus         Zac            Lux      Kaisa   
Partie 2 Pseudo       but ok     Kanuera  Meep Enjooyer      tjfgx   
         Champion      Viego      Khazix         Viktor     Twitch   

                  Blue Support    Red Top Red Jungle     Red Mid  Red Carry  \
Partie   Infos                                                                
Partie 1 Pseudo       Bioniks1     monkaS    N0EQUAL  CG Kinneke     Bendåx   
         Champion       Soraka  Gangplank     Rengar       Yasuo       Ashe   
Partie 2 Pseudo        Vaati73     Lewite  raymmondd     tuctuc1  qfxnhfzkl   
         Champion        Yuumi      Vayne    Nidalee       Akali     Ezreal   

                  Red Support              Outcome  
Partie   Infos                                      
Partie 1 Pseudo       Jonperi  Victoire des Rouges  
         Champion      Maokai  Victoire des Rouges  
Partie 2 Pseudo         Bingu   Victoire des Bleus  
         Champion      Thresh   Victoire des Bleus